In [26]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

In [55]:
db_username = "root"
db_password = ")Q9w8e7r6t5y4u3i2o1p"
db_ip = "localhost"
db_name = 'curriculumchecklist'
db_connection = create_engine("mysql+mysqlconnector://{0}:{1}@{2}/{3}".format(db_username, db_password, db_ip, db_name))

In [86]:
table = pd.read_csv("BITCF.csv")


In [88]:
table.rename(columns={"Course Code": "course_id", "Course Name": "course_name", "Pre-requisite": "course_preq", "Hours": "course_hours",
                      "Units": "course_units", "Year": "course_year", "Semester": "course_sem"}, inplace=True)

In [90]:
program = "BITCF"
relTable = table[["course_id"]].copy()
relTable["program_id"] = program

In [91]:
order = ["course_id", "course_name", "course_hours", "course_units", "course_preq", "course_year", "course_sem"]
table = table[order]

In [94]:
pushCourse = text(""" INSERT INTO courses (course_id, course_name, course_hours, course_units, course_preq, course_year, course_sem)
    VALUES (:course_id, :course_name, :course_hours, :course_units, :course_preq, :course_year, :course_sem)
    ON DUPLICATE KEY UPDATE course_sem = VALUES(course_sem)
""")

pushRel = text(""" INSERT INTO program_courses (course_id, program_id)
               VALUES (:course_id, :program_id)
               """)

data = table.where(pd.notnull(table), None).to_dict(orient='records')
relData = relTable.to_dict(orient="records")
relData



[{'course_id': 'BIT1101', 'program_id': 'BITCF'},
 {'course_id': 'CCS1101', 'program_id': 'BITCF'},
 {'course_id': 'CCS1101L', 'program_id': 'BITCF'},
 {'course_id': 'CCS1102', 'program_id': 'BITCF'},
 {'course_id': 'CCS1102L', 'program_id': 'BITCF'},
 {'course_id': 'FCL100', 'program_id': 'BITCF'},
 {'course_id': 'GEC100', 'program_id': 'BITCF'},
 {'course_id': 'GEC500', 'program_id': 'BITCF'},
 {'course_id': 'GEE300', 'program_id': 'BITCF'},
 {'course_id': 'NST100', 'program_id': 'BITCF'},
 {'course_id': 'PATHFIT1', 'program_id': 'BITCF'},
 {'course_id': 'BIT1202', 'program_id': 'BITCF'},
 {'course_id': 'BIT1202L', 'program_id': 'BITCF'},
 {'course_id': 'BIT1225', 'program_id': 'BITCF'},
 {'course_id': 'BIT1225L', 'program_id': 'BITCF'},
 {'course_id': 'CCS1203', 'program_id': 'BITCF'},
 {'course_id': 'CCS1203L', 'program_id': 'BITCF'},
 {'course_id': 'FCL200', 'program_id': 'BITCF'},
 {'course_id': 'GEC600', 'program_id': 'BITCF'},
 {'course_id': 'GEC800', 'program_id': 'BITCF'},
 {

In [96]:
with db_connection.begin() as conn:
    conn.execute(pushCourse, data)
    conn.execute(pushRel, relData)